# Implementation of a 3-layer neural network
The basic structure of the neural network model is:

input x -> z1 = ReLU(W1\*x+b1) -> z2 = W2\*z1+b2 -> softmax(z2)

The MNIST data from package chainer is used to train and test model

In [1]:
import numpy as np
import chainer

In [2]:
# Define functions
def init_W(shape):
'''
Use small random numbers to initializ matrix W.
Numbers greater than 1 or smaller than -1 are truncated.
'''
    W = np.random.normal(0,0.5,shape)
    W[W > 1] = 1
    W[W < -1] = -1
    return W

def init_b(length):
'''
Use small random numbers to initializ vector b.
Numbers greater than 1 or smaller than -1 are truncated.
'''
    b = np.random.normal(0,0.5,length)
    b[b > 1] = 1
    b[b < -1] = -1
    return b

def softmax(arr):
'''
Calculate softmax for a vector of numbers.
'''
    arr = np.exp(arr)
    return arr / arr.sum()

def foreward(x):
'''
Do the forward calculation with input vector x.
Output softmax result.
'''
    z1 = np.dot(W1, x) + b1
    z1[z1 < 0] = 0
    z2 = np.dot(W2, z1) + b2
    softm = softmax(z2)
    return softm, z1

def PD_softm_z2(softm):
'''
Calculate partial derivative matrix of log(softmax(z2)) to z2.
'''
    a = np.identity(k)
    return (a - np.row_stack([softm]*k))

def get_gradient(x,y):
'''
Calculate gradient for W1, b1, W2, b2
'''
    grad_W1 = np.zeros((L2, L1))
    grad_b1 = np.zeros(L2)
    grad_W2 = np.zeros((k, L2))
    grad_b2 = np.zeros(k)
    for i in range(batch_size):
        softm, z1 = foreward(x[i])
        J_softm_z2 = PD_softm_z2(softm)
        v2 = np.dot(-y[i],J_softm_z2)
        grad_W2 = grad_W2 + ((np.row_stack([z1]*k).T)*v2).T
        grad_b2 = grad_b2 + v2
        v1 = np.dot(v2, W2)
        foo = np.row_stack([x[i]]*L2)
        is_positive = (z1 > 0).astype(float)
        foo = (foo.T*is_positive).T
        grad_W1 = grad_W1 + (foo.T*v1).T
        grad_b1 = grad_b1 + v1*is_positive
    grad_W2 = grad_W2 / batch_size + penalty*W2
    grad_b2 = grad_b2 / batch_size + penalty*b2
    grad_W1 = grad_W1 / batch_size + penalty*W1
    grad_b1 = grad_b1 / batch_size + penalty*b1
    return grad_W2, grad_b2, grad_W1, grad_b1

def evaluate():
'''
Make predictions for test data, check test accuracy.
'''
    y_prediction = []
    for i in range(len(test_label)):
        softm, z1 = foreward(test_arr[i])
        y_prediction.append(np.argmax(softm))
    accurate = [a == b for a,b in zip(test_label,y_prediction)]
    accuracy = sum(accurate) / len(accurate)
    return accuracy

def one_hot(a, k):
'''
Return a vector of length k, with ath element 1 and other 0s
'''
    arr = np.zeros(k)
    arr[a] = 1
    return arr

In [12]:
# Set global parameters
L1 = 784   # length of input vector
L2 = 100   # number of units in hidden layer
k = 10     # number of class to predict
batch_size = 10  # batch size
n_batch = 60000 // batch_size  # number of batches for an epoch
penalty = 0.0001   # penalty parameter, lambda
eta = 0.08  # learning rate

In [9]:
# Read in data from chainer
train, test = chainer.datasets.get_mnist()
train_arr = []
train_label = []
test_arr = []
test_label = []
for i in range(len(train)):
    train_arr.append(train[i][0])
    train_label.append(one_hot(train[i][1],k))
for i in range(len(test)):
    test_arr.append(test[i][0])
    test_label.append(test[i][1])
train_arr = np.array(train_arr)
train_label = np.array(train_label)
test_arr = np.array(test_arr)

In [13]:
# Initialize W1, b1, W2, b2
W1 = init_W((L2, L1))
b1 = init_b(L2)
W2 = init_W((k, L2))
b2 = init_b(k)

In [14]:
# Start training
accuracy = []
for epoch in range(20):
    shuffle = list(range(60000))
    np.random.shuffle(shuffle)
    for i in range(n_batch):
        x = train_arr[shuffle[i*batch_size:(i+1)*batch_size]]
        y = train_label[shuffle[i*batch_size:(i+1)*batch_size]]
        grad_W2, grad_b2, grad_W1, grad_b1 = get_gradient(x,y)
        W2 = W2 - eta*grad_W2
        b2 = b2 - eta*grad_b2
        W1 = W1 - eta*grad_W1
        b1 = b1 - eta*grad_b1
    acc_epoch = evaluate()
    accuracy.append(acc_epoch)
    print('epoch '+str(epoch+1)+' accuracy = ', acc_epoch)

epoch 1 accuracy =  0.9308
epoch 2 accuracy =  0.9499
epoch 3 accuracy =  0.9567
epoch 4 accuracy =  0.9589
epoch 5 accuracy =  0.9623
epoch 6 accuracy =  0.9648
epoch 7 accuracy =  0.967
epoch 8 accuracy =  0.969
epoch 9 accuracy =  0.97
epoch 10 accuracy =  0.9695
epoch 11 accuracy =  0.9711
epoch 12 accuracy =  0.9718
epoch 13 accuracy =  0.9722
epoch 14 accuracy =  0.9733
epoch 15 accuracy =  0.9727
epoch 16 accuracy =  0.9751
epoch 17 accuracy =  0.9736
epoch 18 accuracy =  0.9742
epoch 19 accuracy =  0.9753
epoch 20 accuracy =  0.9758
